In [1]:
import sys
sys.path.insert(0, "../../src")
from pathlib import Path
from collections import Counter

import numpy as np
from scipy.special import softmax

from gen.util import read_data, write_jsonl
from rte.aggregate import generate_micro_macro_df

In [2]:
root_data = Path("../../data").resolve()
root_model = Path("../../models").resolve()

In [3]:
# constants
LOOKUP = {
    "verifiable": {"no": "NOT VERIFIABLE", "yes": "VERIFIABLE"},
    "label": {"nei": "NOT ENOUGH INFO", "r": "REFUTES", "s": "SUPPORTS"}
}

SEED = 123456789

LABEL2ID = {"SUPPORTS": 0, "NOT ENOUGH INFO": 1, "REFUTES": 2}
ID2LABEL = {0: "SUPPORTS", 1: "NOT ENOUGH INFO", 2: "REFUTES"}

# Init

In [4]:
import evaluate
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments, 
    Trainer,
    DataCollatorWithPadding,
    TextClassificationPipeline,
    pipeline
)
import torch
torch.backends.cuda.matmul.allow_tf32 = True

/scratch/users/k21190024/envs/conda/mainenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Huggingface Init

## Model

In [5]:
accuracy_metric = evaluate.load("accuracy")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")
f1_metric = evaluate.load("f1")

In [6]:
model_checkpoint = "xlnet-base-cased"
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        model_checkpoint, 
        num_labels=3, 
        id2label=ID2LABEL, 
        label2id=LABEL2ID
    )

model = model_init()
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, do_lower_case=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def preprocess(examples):
    return tokenizer(examples["evidence"], examples["claim"])

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    results = {}
    results.update(accuracy_metric.compute(predictions=predictions, references=labels))
    results.update(recall_metric.compute(predictions=predictions, references=labels, average="macro"))
    results.update(precision_metric.compute(predictions=predictions, references=labels, average="macro"))
    results.update(f1_metric.compute(predictions=predictions, references=labels, average="macro"))
    
    return results

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [7]:
dataset = ["fever", "climatefeverpure", "fever-climatefeverpure", "climatefever", "fever-climatefever"]
doc_sent = ["doc", "sent"]

di = 2
ds = 1

model_store_path = root_model.joinpath("sentence-models" if ds == 1 else "document-models")
model_store_path.mkdir(exist_ok=True)
model_store_path = model_store_path / f"{dataset[di]}-{model_checkpoint}-{doc_sent[ds]}"

## Dataset

In [8]:
datap = root_data / f"{doc_sent[ds]}-dataset"

data = DatasetDict({
    "train": Dataset.from_list(read_data(datap / f"{dataset[di]}.train.n5.jsonl")),
    "validation": Dataset.from_list(read_data(datap / f"{dataset[di]}.dev.n5.jsonl")),
    "fever_test": Dataset.from_list(read_data(datap / f"{dataset[0]}.test.n5.jsonl")),
    "climatefever_test": Dataset.from_list(read_data(datap / f"{dataset[1]}.test.n5.jsonl"))
}).map(preprocess, batched=True)

## Trainer

In [8]:
batch_size = 16
learning_rate = 4e-4
epoch = 4
metric_name = "f1"
warmup_ratio=0.1
save_steps=200
eval_steps=200

# Hyperparameter tuning

In [9]:
# shard the data if the dataset is large for hyperparameter tuning
shard = data["train"].num_rows > 50000
hp_tune_train = data["train"].shuffle(seed=SEED).shard(num_shards=5, index=1)

In [10]:
training_args = TrainingArguments(
    model_store_path,
    overwrite_output_dir=True,
    evaluation_strategy = "steps",
    eval_steps=eval_steps,
    save_strategy = "no",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=0.01,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    seed=SEED,
    data_seed=SEED,
    warmup_ratio=warmup_ratio,
    report_to="tensorboard"
)

_ = model.train()
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=hp_tune_train if shard else data["train"],
    eval_dataset=data["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [11]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_categorical("learning_rate", [1e-5, 3e-5, 2e-5]),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 5),
    }

def compute_objective(metrics):
    return metrics["f1"]

In [12]:
best_run = trainer.hyperparameter_search(
    direction="maximize", 
    n_trials=20, 
    hp_space=optuna_hp_space
)

[I 2023-07-03 13:02:16,683] A new study created in memory with name: no-name-0ca0b7ce-7d62-488a-b0b3-121379fdd2e1
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'lo

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.716896,0.687972,0.667642,0.787427,0.616972
400,No log,0.377311,0.861386,0.858889,0.856307,0.854226
600,0.552400,0.335807,0.878376,0.872998,0.878234,0.870160
800,0.552400,0.339101,0.880427,0.877289,0.878673,0.874102
1000,0.261400,0.283244,0.897885,0.892081,0.893379,0.891712
1200,0.261400,0.325783,0.889976,0.890187,0.885488,0.885494
1400,0.261400,0.295013,0.896596,0.894413,0.891339,0.891438


[I 2023-07-03 13:11:32,540] Trial 0 finished with value: 3.573787006087937 and parameters: {'learning_rate': 1e-05, 'per_device_train_batch_size': 32, 'num_train_epochs': 1}. Best is trial 0 with value: 3.573787006087937.
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.991626,0.542914,0.527758,0.573669,0.431203
400,No log,0.560934,0.761146,0.745409,0.792976,0.727212
600,0.760900,0.395433,0.850021,0.839189,0.858914,0.835378
800,0.760900,0.367791,0.870233,0.863724,0.873036,0.861422
1000,0.307600,0.302653,0.894428,0.887027,0.891225,0.887340
1200,0.307600,0.318581,0.892730,0.892704,0.887653,0.888290
1400,0.307600,0.284869,0.900580,0.894967,0.898088,0.894205
1600,0.239300,0.314854,0.892905,0.893231,0.887952,0.888542
1800,0.239300,0.279520,0.904915,0.900820,0.900270,0.899611
2000,0.193400,0.342002,0.895190,0.894341,0.893667,0.890263


[I 2023-07-03 13:58:01,520] Trial 1 finished with value: 3.658646782667355 and parameters: {'learning_rate': 1e-05, 'per_device_train_batch_size': 32, 'num_train_epochs': 5}. Best is trial 1 with value: 3.658646782667355.
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly

...

[I 2023-07-03 16:06:20,907] Trial 17 pruned. 
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stre

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.733386,0.638877,0.610234,0.735968,0.508581
400,No log,0.352569,0.862148,0.853291,0.858876,0.851588


[I 2023-07-03 16:08:58,443] Trial 18 pruned. 
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stre

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,1.072086,0.448943,0.457304,0.403392,0.356058


/scratch/users/k21190024/envs/conda/p-dis-torch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023-07-03 16:09:56,131] Trial 19 pruned. 


In [13]:
best_run

BestRun(run_id='1', objective=3.658646782667355, hyperparameters={'learning_rate': 1e-05, 'per_device_train_batch_size': 32, 'num_train_epochs': 5}, run_summary=None)

## Train with best hyperparameters

In [ ]:
training_args = TrainingArguments(
    model_store_path,
    overwrite_output_dir=True,
    evaluation_strategy = "steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=5,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    seed=SEED,
    data_seed=SEED,
    warmup_ratio=warmup_ratio,
    report_to="tensorboard",
    tf32=True
)

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=data["train"],
    eval_dataset=data["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

for n, v in best_run.hyperparameters.items():
    if n == "per_device_train_batch_size" and v == 32:
        # out-of-memory not caught in hyperparameter tuning but
        # when whole data was used, OOM was thrown
        setattr(trainer.args, "per_device_train_batch_size", 4)
        setattr(trainer.args, "gradient_accumulation_steps", 8)
        setattr(trainer.args, "per_device_eval_batch_size", v)
    else:
        setattr(trainer.args, n, v)
    
trainer.train()

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.weight', 'logits_proj.weight', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1000,0.529300,0.357253,0.864400,0.857064,0.866255,0.856628
2000,0.228600,0.250765,0.912420,0.910470,0.909964,0.909361
3000,0.187700,0.215208,0.921096,0.918998,0.917544,0.918153
4000,0.168400,0.239042,0.921205,0.920903,0.920024,0.919108
5000,0.154700,0.236894,0.924752,0.924051,0.922165,0.922694
6000,0.145800,0.196096,0.937957,0.935437,0.936045,0.935508
7000,0.140800,0.245169,0.926443,0.927014,0.924865,0.924973
8000,0.135800,0.187714,0.938448,0.936924,0.936070,0.936390
9000,0.108800,0.216704,0.941340,0.939451,0.940377,0.939269
10000,0.115900,0.210406,0.939649,0.938938,0.937430,0.937756


In [32]:
trainer.save_model(model_store_path.parent / (model_store_path.stem + ".out"))

# Evaluate

In [25]:
import pandas as pd
from sklearn.metrics import classification_report

## Test on validation data

In [26]:
preds = trainer.predict(data["validation"])
micro_val, macro_val = generate_micro_macro_df(data["validation"], preds)

In [27]:
print(classification_report(y_true=micro_val["actual"], y_pred=micro_val["predicted"]))

                 precision    recall  f1-score   support

NOT ENOUGH INFO       0.98      0.95      0.97      7141
        REFUTES       0.94      0.93      0.94      5479
       SUPPORTS       0.91      0.96      0.94      5706

       accuracy                           0.95     18326
      macro avg       0.95      0.95      0.95     18326
   weighted avg       0.95      0.95      0.95     18326



## Test on test data

### FEVER

In [28]:
preds = trainer.predict(data["fever_test"])
micro_val, macro_val = generate_micro_macro_df(data["fever_test"], preds)

In [29]:
print(classification_report(y_true=micro_val["actual"], y_pred=micro_val["predicted"]))

                 precision    recall  f1-score   support

NOT ENOUGH INFO       0.99      0.98      0.98      6666
        REFUTES       0.95      0.91      0.93      5272
       SUPPORTS       0.92      0.96      0.94      5389

       accuracy                           0.95     17327
      macro avg       0.95      0.95      0.95     17327
   weighted avg       0.95      0.95      0.95     17327



### Climate-FEVER

In [30]:
preds = trainer.predict(data["climatefever_test"])
micro_val, macro_val = generate_micro_macro_df(data["climatefever_test"], preds)

In [31]:
print(classification_report(y_true=micro_val["actual"], y_pred=micro_val["predicted"]))

                 precision    recall  f1-score   support

NOT ENOUGH INFO       0.74      0.55      0.63       235
        REFUTES       0.43      0.48      0.46        48
       SUPPORTS       0.61      0.80      0.69       176

       accuracy                           0.64       459
      macro avg       0.59      0.61      0.59       459
   weighted avg       0.66      0.64      0.64       459

